In [1]:
import numpy as np

def Grad(f, x0, h=1e-6, i=-1):
    """
    Función que calcula el Grad de una función en un punto
    """
    n = len(x0)
    if i in range(n):
        z = np.zeros(n)
        z[i] = h/2
        Grad = (f(x0 + z) - f(x0 - z))/h
    else:
        Grad = np.zeros(n)
        for j in range(n):
            z = np.zeros(n)
            z[j] = h/2
            Grad[j] = (f(x0 + z) - f(x0 - z))/h
    return np.array(Grad)


def Hess(f, x0, h=1e-4, method="basic"):
    """
    Función que calcula la Hessiana  de una función en un punto.
    f: función sobre la cual queremos calcular la hessiana.
    x0: Punto sobre el cual queremos hacer el cálculo
    h: nivel de precisión para hacer el cálculo
    method: Método por el cual se quiere hacer puede ser:
             'basic', 'grad', 'centered', 'gradCentered'
    """
    n = len(x0)
    Hess = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            z_i = np.zeros(n)
            z_i[i] = h
            z_j = np.zeros(n)
            z_j[j] = h
            if method == "basic":
                Hess[i, j] = (f(x0 + z_j + z_i) - f(x0 + z_i) -
                              f(x0+z_j) + f(x0)) / (h**2)
            elif method == "grad":
                Hess[i, j] = (Grad(f, x0+z_j, h, i) - Grad(f, x0, h, i) +
                              Grad(f, x0+z_i, h, j) - Grad(f, x0, h, j))/(2*h)
            elif method == "centered":
                if i == j:
                    Hess[i, j] = (-f(x0+2*z_i) + 16*f(x0+z_i) - 30*f(x0) +
                                  16*f(x0-z_i) - f(x0-2*z_i)) / (12*h**2)
                else:
                    Hess[i, j] = (f(x0+z_i+z_j) - f(x0 + z_i - z_j) -
                                  f(x0 - z_i + z_j) + f(x0-z_i-z_j))/(4*h**2)
            elif method == "gradCentered":
                Hess[i, j] = (Grad(f, x0+z_j, h)[i] - Grad(f, x0-z_j, h)[i] +
                              Grad(f, x0+z_i, h)[j] - Grad(f, x0-z_i, h)[j])\
                               / (4 * h)
    return Hess



In [2]:
def condiciones_wolfe(f, x0, alpha, pk, c1=1e-4, c2=1e-2, tol=1e-5):
    """
    Función que evalúa las condiciones de wolfe para una alpha.
    f:  función que optimizamos
    x0: punto anterior un numpy.array
    alpha: valor que cumplirá condiciones de wolfe.
    pk: dirección de decenso un numpy.array
    """
    def grad(alpha): return Grad(f, x0+alpha*pk, tol)
    def phi(alpha): return f(x0 + alpha*pk)  # Ojo que phi(0) = f(x0)
    def linea(alpha): return phi(0) + c1 * alpha * np.dot(g_x0, pk)
    g_x0 = grad(0)  # grad(0) = Grad(f,x0)
    cond_1 = linea(alpha) - phi(alpha) >= 0
    cond_2 = np.dot(grad(alpha), pk) - c2 * np.dot(g_x0, pk) >= 0
    return cond_1 and cond_2

def genera_alpha(f, x0, pk, c1=1e-4, c2 = 0.5, tol=1e-5):
    """
    Backtracking LS i.e. Algoritmo que encuentra una
    alpha que cumpla condiciones de wolfe.
    """
    alpha, rho = 1, 3/4
    Gkpk = Grad(f, x0).dot(pk)
    while f(x0 + alpha*pk) > f(x0) + c1*alpha*Gkpk:
        alpha *= rho
    return alpha

In [3]:
def modificacion_hessiana(Hessiana, lam=0.5):
    while not is_pos_def(Hessiana):
        Hessiana = Hessiana + lam*np.eye(len(Hessiana))
    return Hessiana

In [4]:
def generar_conjunto_canonico(n):
    A = np.array(np.eye(n))
    return [A[i] for i in range(n)]

n=5
A = np.matrix(np.eye(n)).T
b = np.zeros(n)+1
cc = generar_conjunto_canonico(n)
print(cc)
cc[0]

[array([1., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0.]), array([0., 0., 1., 0., 0.]), array([0., 0., 0., 1., 0.]), array([0., 0., 0., 0., 1.])]


array([1., 0., 0., 0., 0.])

## Gradiente Conjugado (40 puntos)

In [5]:
def direcciones_conjugadas(x0, cc, A, b):
    xk = x0
    for k in range(len(x0)):
        rk  = np.array(A.dot(xk)-b) 
        alpha = np.dot(-rk,cc[k])/(np.dot(cc[k],A.dot(cc[k])))
        xk_1 = xk + alpha*cc[k] 
    return xk_1

n=5
A = np.array(np.eye(n)) # hacerlo con matriz rala
b = np.zeros(n)+1
cc = generar_conjunto_canonico(n)
x0 = np.array(np.zeros(n))
direcciones_conjugadas(x0, cc, A, b)
print(direcciones_conjugadas(x0, cc, A, b))
#print(direcciones_conjugadas(x0,cc,A,b))

[0. 0. 0. 0. 1.]


## pruebas con libreria python

In [23]:
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

In [24]:
def direcciones_conjugadas_ralas(x0, cc, A, b):
    A_sparce = csr_matrix(A)
    xk = x0
    for k in range(len(x0)):
        Axk = A_sparce.dot(xk)
        rk  = Axk-b 
        alpha = np.dot(-rk,cc[k])/(np.dot(cc[k],A_sparce.dot(cc[k]))) 
        xk_1 = xk + alpha*cc[k] 
    return xk_1

n=5
A = np.array(np.eye(n)) 
b = np.zeros(n)+1
cc = generar_conjunto_canonico(n)
x0 = np.array(np.zeros(n))
print(sparse.csr_matrix(A))
direcciones_conjugadas(x0, cc, A, b)
print(direcciones_conjugadas(x0, cc, A, b))

  (0, 0)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 4)	1.0
[0. 0. 0. 0. 1.]


In [8]:
def gradiente_conjugado(x0, A, b): #Algoritmo 5.2
    xk = x0
    b = np.matrix(b).T
    rk = np.dot(A, x0) - b
    pk = -rk
    while not (rk.T * rk ==  0):
        alphak = rk.T * rk / (pk.T * A * pk)
        alphak= alphak[0,0]
        xk_1 = xk + alphak * pk
        rk_1 =  rk + alphak * A * pk
        betak_1 = (rk_1.T * rk_1) / (rk.T * rk)
        betak_1 = betak_1[0,0]
        pk_1 = -rk_1 + betak_1 * pk
        xk, rk, pk = xk_1, rk_1, pk_1
    return xk

def gradiente_conjugado_precond(x0, A, b, M):
    xk = x0
    b = np.matrix(b).T
    rk = np.dot(A, x0) - b
    yk = np.linalg.solve(M, rk)
    pk = -yk 
    while not (rk.T * rk ==  0):
        alphak = rk.T * yk / (pk.T * A * pk)
        alphak= alphak[0,0]
        xk_1 = xk + alphak * pk
        rk_1 =  rk + alphak * A * pk
        yk_1 = np.linalg.solve(M, rk_1)
        betak_1 = (rk_1.T * rk_1) / (rk.T * rk)
        betak_1 = betak_1[0,0]
        pk_1 = -yk_1 + betak_1 * pk
        xk, rk, pk, yk  = xk_1, rk_1, pk_1, yk_1
    return xk

n=15
A = np.matrix(np.eye(n)).T
b = np.zeros(n)+1
x0 = np.matrix(np.zeros(n)).T
print(gradiente_conjugado_precond(x0, A, b, A))

[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [9]:
#para darme una idea

from scipy import sparse
A = np.array([[1, 0, 0, 1, 0, 0], [0, 0, 2, 0, 0, 1],[0, 0, 2, 0, 0, 1], [0, 0, 0, 2, 0, 0]])
print("Original Matrix: \n", A)
S = sparse.csr_matrix(A)
print("Sparse Matrix: \n",S)
print("shape de S",S.shape)
print("Original Matrix otra vez: \n",S.todense())

Original Matrix: 
 [[1 0 0 1 0 0]
 [0 0 2 0 0 1]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]
Sparse Matrix: 
   (0, 0)	1
  (0, 3)	1
  (1, 2)	2
  (1, 5)	1
  (2, 2)	2
  (2, 5)	1
  (3, 3)	2
shape de S (4, 6)
Original Matrix otra vez: 
 [[1 0 0 1 0 0]
 [0 0 2 0 0 1]
 [0 0 2 0 0 1]
 [0 0 0 2 0 0]]


In [10]:
import random
random.seed(175311) #  Cambien a su propia clave
Diag_A = [random.randint(1,1000) for x in range(1000000)] #no hay ceros
print(len(Diag_A))
no_cero = len(Diag_A)
b = [random.randint(1,1000) for x in range(1000000)] #lista

1000000


In [11]:
#en este caso la matriz rala es una matriz diagonal cuyas entradas en la
#diagonal son distintas de cero
Diagonal = np.diag(Diag_A) # esta es la matriz rala
#print(Diagonal.shape)matrix de 1000000x1000000(por eso se trabó mi compu >.<)

In [19]:
#guardamos las entradas distintas de cero en una matriz 
rows, cols = (no_cero,1 )
compacta= [[0 for i in range(cols)] for j in range(rows)] #una lista como b
print(compacta[0][0])

0


In [20]:
def matrix_compacta(matriz_rala):
    """Input: matriz rala (diagonal con entradas distintas de cero) 
        Output: lista con las entradas distintas de cero
        en este caso son 10000 entradas distintas de cero
    """
    rows, cols = (no_cero,1) 
    compacta  = [[0 for i in range(1)] for j in range(rows)] #inicializar
    k = 0
    for i in range(1000000):#renglon
        for j in range(1000000):#columna
            if (matriz_rala[i][j] != 0):
                compacta[k][0] = matriz_rala[i][j]#guardamos ese elemento
                k = k+1
    return compacta

In [21]:
#definimos operaciones con estas matrices compactas
def dot_product_rala(matriz_rala,b):
    """ Input: 
    matriz_rala:matriz rala (diagnoal jeje)
    b: vector """
    compacta = np.array(matrix_compacta(matriz_rala))
    arr = np.array(b)
    return np.dot(compacta,arr)

In [25]:
dot_product_rala(Diagonal,b) #lo interrumpi

KeyboardInterrupt: 

In [ ]:
def gradiente_conjugado_precond_mirala(x0, Diagonal, b, M):
    """
    x0: vector
    Diagnoal = matriz Rala 
    b = lista 
    M matriz
    """
    xk = x0
    
    b = np.array(b)
    rk = dot_product_rala(Diagonal,b) - b
    yk = np.linalg.solve(M, rk)
    pk = -yk 
    while not (rk.T * rk ==  0):
        alphak = rk.T * yk / (pk.T * A * pk)
        alphak= alphak[0,0]
        xk_1 = xk + alphak * pk
        rk_1 =  rk + alphak * dot_product_rala(Diagonal,(dot_product_rala(Diagonal,pk)))
        yk_1 = np.linalg.solve(M, rk_1)
        betak_1 = (rk_1.T * rk_1) / (rk.T * rk)
        betak_1 = betak_1[0,0]
        pk_1 = -yk_1 + betak_1 * pk
        xk, rk, pk, yk  = xk_1, rk_1, pk_1, yk_1
    return xk

x0 = np.matrix(np.zeros(n)).T
M = np.matrix(np.eye(n)).T
print(gradiente_conjugado_precond(x0, Diagonal, b, M))


## DFP (20 puntos)

In [48]:
import numpy as np
from numpy import linalg as LA

def DFP_Bk(yk, sk, Bk):
    """
    Función que calcula La actualización DFP de la matriz Bk
    In:
      yk: Vector n
      sk: Vector n
      Bk: Matriz nxn
    Out:
      Bk+1: Matriz nxn
    """
    n = len(yk)
    rhok = 1 / (yk.T*sk)
    Vk = (np.eye(n) - rhok * yk*sk.T)
    Bk1 = Vk * Bk * Vk.T + rhok * yk * yk.T
    return Bk1


def DFP_Hk(yk, sk, Hk):
    """
    Función que calcula La actualización DFP de la matriz Hk
    In:
      yk: Vector n
      sk: Vector n
      Hk: Matriz nxn
    Out:
      Hk+1: Matriz nxn
    """
    yk = np.array([yk]).T
    sk = np.array([sk]).T
    Hk1 = Hk - (Hk * yk.dot(yk.T) * Hk)/(yk.T * Hk * yk) + sk.dot(sk.T)/yk.T.dot(sk)
    return Hk1


def BFGS_Hk(yk, sk, Hk):
    """
    Función que calcula La actualización BFGS de la matriz Hk
    In:
      yk: Vector n
      sk: Vector n
      Hk: Matriz nxn
    Out:
      Hk+1: Matriz nxn
    """
    n = len(yk)
    yk = np.array([yk]).T
    sk = np.array([sk]).T
    rhok = 1 / yk.T.dot(sk)
    Vk = (np.eye(n) - rhok * yk.dot(sk.T))
    Hk1 = Vk.T * Hk * Vk + rhok * sk.dot(sk.T)
    return Hk1


def BFGS_Bk(yk, sk, Bk):
    """
    Función que calcula La actualización BFGS de la matriz Bk
    In:
      yk: Vector n
      sk: Vector n
      Bk: Matriz nxn
    Out:
      Bk+1: Matriz nxn
    """
    return Bk - (np.dot(Bk, np.dot(sk, np.dot(sk, Bk)))) / (np.dot(sk, np.dot(Bk, sk))) + np.dot(yk, yk) / np.dot(yk, sk)



In [49]:
def BFGS(f, x0, tol, H0, maxiter=10000):
    k = 0
    Gk = Grad(f, x0)
    Hk = H0
    xk = np.array(x0)
    xk1 = np.array(x0)
    sk = np.array(100)
    while (LA.norm(Gk) > tol and LA.norm(sk) > tol and k <= maxiter):
        pk = - Hk.dot(Gk)
        alphak = genera_alpha(f, xk, pk)
        xk1 = xk + alphak * pk
        sk = xk1 - xk
        Gk1 = Grad(f, xk1)
        yk = Gk1 - Gk
        Hk = DFP_Hk(yk, sk, Hk)
        k += 1
        xk = xk1
        Gk = Gk1
    return xk1, k

def cuadrados(x):
    resultado=0
    for i in range(len(x)):
        resultado += x[i]**2
    return resultado


x0 = [(-1)**i*10 for i in range(10) ]
B0 = Hess(cuadrados,x0)
H0 = LA.inv(B0)
x, k = BFGS(cuadrados, x0, 1e-15, H0)
print(f'Llegué a {x} en {k} iteraciones')

Llegué a [-1.42103940e-04 -1.42106305e-04 -1.42103940e-04 -1.42106305e-04
 -1.42103940e-04 -1.42106305e-04  2.84237581e-05  2.84210696e-05
  8.52672934e-05  8.52636356e-05] en 72 iteraciones
